# Using Jupyter notebooks with R in the classroom

<div style="display:flex;margin: 20px 10%">
<img src="https://avatars1.githubusercontent.com/u/7388996?v=3&s=400" alt="Jupyter logo" style="height:200px"/>
<div style="font-size:300%;margin-top:10%;width:42px">+</div>
<img src="https://www.r-project.org/logo/Rlogo.png" alt="R logo" style="height:150px"/>
</div>
<div style="text-align:center;width:100%;">
useR!2016<br/>
Tanya Schlusser
</div>

## Motivation: work at Imperial College London

* A new introductory programming course for undergraduate geologists (2010)
* Students are _not_ computer science majors
* A 5 year effort to find the most successful approach
* C. T. Jacobs, G. J. Gorman, H. E. Rees, L. E. Craig (In Press)


## Most successful teaching method

* 'Flipped classroom' format -- read material beforehand with most of class devoted to worked examples
* 10-15 minute bite-sized lectures with a worked example
* 30 minute in-class exercise
* Repeat (for the 3 hour course)

## In-class teaching

* Grad students were on hand in class, the ratio was about 10:1 students-to-TAs
* Students used sticky notes — red and green — to signal they were done or needed help
* Exercises were presented in Jupyter notebooks

<div style="display:flex;width:80%">
<img src="https://avatars1.githubusercontent.com/u/7388996?v=3&s=400" alt="Jupyter logo" style="height:200px;"/>
<div style="width:50%;font-size:200%;line-height:180%">
Try it out: <br/>
<a href="http://jupyter.rocks">http://jupyter.rocks</a>
</div>
</div>

# Benefits

- Libraries and data sets are preloaded
- Simple questions can be answered immediately in class
- Students can help each other

# Caveats

This is a presentation environment, not a coding one.

- Don't do short examples, do long exercises like with Jacobs _et al._ <br/><small>(or students just click and run without understanding)</small>
- Use RStudio or plain R for work <br/><small>(or students might have trouble understanding the sequential nature of coding)</small>
- The paper had a 1:10 TA - to - student ratio in class. <br/><small>(But these were first-year undergraduates.)</small> 

# What you need to do this yourself

- To be able to create the notebook
- To be able to serve the notebook

## Creating the notebook

<div style="display:flex;width:100%;padding:0">
<div style="width:300px;overflow:hidden;float:right"><img src="https://upload.wikimedia.org/wikipedia/en/thumb/c/cd/Anaconda_Logo.png/200px-Anaconda_Logo.png" alt="Anaconda Logo" style="margin:20px;padding-left:80px"/>
</div>

<ul>
<li>Anaconda is a commercial Python redistribution</li>
<li>Jupyter is a Python application, and is bundled with Anaconda</li>
<li>It also provides a repository for popular R libraries</li>
</ul>

</div>



### Quick details for creating the notebook

1. First, install Anaconda (anaconda.org)
2. Next, create the environment. (It will be isolated from your regular R environment.) In a terminal shell, type:
```shell
$ conda create -n jupyter_IRKernel python=3 anaconda
$ conda install -c r -n jupyter_IRKernel r-essentials
```
3. When you want to use the environment, activate it:
```shell
$ source activate jupyter_IRKernel
$ jupyter notebook  # ... and start a local server
```

# Create the server

<div style="display:flex;width:80%;margin:30px;">
<img src="https://www.docker.com/sites/default/files/legal/small_v.png" alt="Docker logo" style="height:180px"/>
<span style="font-size:300%;margin-top:10%">+</span>
<img src="https://avatars1.githubusercontent.com/u/7388996?v=3&s=400" alt="Jupyter logo" style="height:160px"/>
</div>

## Quick details for serving your notebook

1. Install Docker on a Linux-like machine: https://docs.docker.com/engine/installation
2. Add your notebook(s) and data to a Dockerfile that starts from Jupyter's pre-built image with IRkernel, and build the Docker Image.
3. Build the image.
4. Run!  ... The commands are long for a slide, but they're also here:<br/>
https://github.com/jupyter/tmpnb-deploy

### Step 1: Install Docker -- the same as in the link

```shell
sudo apt-get update
sudo apt-get install -y apt-transport-https ca-certificates
sudo apt-key adv --keyserver hkp://p80.pool.sks-keyservers.net:80 --recv-keys 58118E89F3A912897C070ADBF76221572C52609D
echo "deb https://apt.dockerproject.org/repo debian-jessie main" > docker.list
sudo mv docker.list /etc/apt/sources.list.d/docker.list
sudo apt-get update
sudo apt-get install -y docker-engine
sudo service docker start
```


I'm just showing it for completeness.

### Step 2:  make a Docker image with your notebook

#### The full Dockerfile for today's example

```shell
FROM jupyter/r-notebook

# Add my own R file
COPY jacobs2016data_0.1.tar.gz /home/$NB_USER/work

# Add the notebook
COPY example-Jupyter-R.ipynb  /home/$NB_USER/work/notebooks

RUN cd /home/$NB_USER/work \
    && R CMD INSTALL jacobs2016data_0.1.tar.gz \
    && rm jacobs2016data_0.1.tar.gz 
```

### Step 2 (continued...):  make a Docker image with your notebook

#### The command to build the Docker Image 

From within the directory containing the Dockerfile and my Jupyter notebook:

```shell
$ docker build  --tag <dockerhub-id>/<image-name>  .
$ # e.g.
$ docker build --tag tanya/r-notebook .
```


### Step 3: here's the whole command I used, verbatim

It's almost the same as on https://github.com/jupyter/tmpnb

<small>
```shell
export TOKEN=$( head -c 30 /dev/urandom | xxd -p )

### Start the redirection server to create temporary URLs
sudo docker run --net=host -d -e CONFIGPROXY_AUTH_TOKEN=$TOKEN  \
 --name=proxy jupyter/configurable-http-proxy  \
 --default-target http://127.0.0.1:9999

### Start the actual notebook server which spawns new temporary notebooks
sudo docker run --net=host -d -e CONFIGPROXY_AUTH_TOKEN=$TOKEN   \
  -v /var/run/docker.sock:/docker.sock  jupyter/tmpnb python orchestrate.py  \
  --image='tanya/r-notebook-example'  \
  --redirect-uri='notebooks/example-Jupyter-R.ipynb'  \
  --pool-size=8  \
  --command="jupyter notebook notebooks/example-Jupyter-R.ipynb --NotebookApp.base_url={base_path} --ip=0.0.0.0 --port {port}"
```
</small>

# Takeaways

* Bite-sized teaching alternating with hands-on work in class yields good performance _and_ student satisfaction
* Jupyter notebooks served on JupyterHub or (like now) using tmpnb can be used to provide a prepared, platform-independent coding environment

# Thank you.  Related links:

## References
* C. T. Jacobs, G. J. Gorman, H. E. Rees, L. E. Craig (In Press). Experiences with efficient methodologies for teaching computer programming to geoscientists. Journal of Geoscience Education. Pre-print: http://arxiv.org/abs/1505.05425
* Related blog post: http://software-carpentry.org/blog/2016/06/teaching-python-to-undergraduate-geoscientists.html
* Related slides: http://christianjacobs.uk/hen-2016-sticky-notes

## Documentation
* Jupyter main page: http://jupyter.org/
* IRkernel main page: https://irkernel.github.io/
* JupyterHub tutorial: http://jupyterhub-tutorial.readthedocs.io/en/latest/JupyterHub-Tutorial.html
* tmpnb GitHub page: https://github.com/jupyter/tmpnb

## Resources
* The executable notebook from this talk: http://54.82.252.54:8000/
* The GitHub repo behind the slides: https://github.com/tanyaschlusser/Jupyter-with-R
* Today's slides: https://tanyaschlusser.github.io/Jupyter-with-R/
* Using Anaconda with R: https://www.continuum.io/conda-for-r
* The Anaconda R repository: https://anaconda.org/r